In [ ]:
import requests
import json
import os
from datetime import datetime
import pandas as pd
from pandas import json_normalize
import geopandas as gpd
from shapely.geometry import Point, LineString
from dotenv import load_dotenv
load_dotenv(override=True)

/Users/kristian/Desktop/os2iot-lorawan-signal-quality/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


True

## Analyse af lora-dækning

- for hver application - find alle devices (fordelt på to Organisationer i OS2IOT)
- for hver device, hent location og søg igennem rxinfo-objektet. Hent SNR, RSSI, location og gatewayID, dr (data rate) og spreadingfactor
- lav en CSV-fil, der for hvert objekt i rxinfo har en LineString i WKT-format:

Se eksempel på 
- LINESTRING (0 0, 0 1) - https://gis.stackexchange.com/a/344885 
- https://github.com/ITKCityLab/OS2IOT-monitoring/blob/main/gateways.py 
- https://aarhuskommune.sharepoint.com/:u:/t/KompetencecenterforIoT/EUrWcv0QBMRMpRugatQFjFQBApI7SGllasdaAsC2OYiDwA?e=VgN73Q

In [1]:
base_url = os.getenv('os2iot_BASE_URL')
os2iot_api_1 = os.getenv('os2iot_api_1')
os2iot_api_2 = os.getenv('os2iot_api_2')

urls = [
  f"{base_url}/application?limit=99999&offset=0",
  f"{base_url}/application?limit=99999&offset=0"
]

api_keys = [os2iot_api_1, os2iot_api_2]

all_data_1 = []
all_data_2 = []

for url, api_key in zip(urls, api_keys):
  headers = {
    'x-api-key': api_key
  }
  while url:
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
      data = json.loads(response.text)
      if api_key == api_keys[0]:
        all_data_1.extend(data['data'])
      else:
        all_data_2.extend(data['data'])
      if 'next' in data:
        url = data['next']
      else:
        url = None
    else:
      print(f"Failed to fetch data from {url}")
      break

data_1 = {'data': all_data_1}
data_2 = {'data': all_data_2}

aids_1 = [item['id'] for item in data_1['data']]
aids_2 = [item['id'] for item in data_2['data']]

results = []

for aids, api_key in zip([aids_1, aids_2], api_keys):
  for id in aids:
    headers = {
      'x-api-key': api_key
    }
    new_url = f"{base_url}/application/{id}/iot-devices?limit=99999&offset=0"
    while new_url:
      new_response = requests.get(new_url, headers=headers)
      if new_response.status_code == 200:
        try:
          new_data = json.loads(new_response.text)
          results.append(new_data)
          # Check if there is a next page
          if 'next' in new_data:
            new_url = new_data['next']
          else:
            new_url = None
        except json.JSONDecodeError:
          print(f"Error decoding JSON for id {id} with API key {api_key}")
          new_url = None
      else:
        print(f"Failed to fetch data for id {id} with API key {api_key}")
        new_url = None

NameError: name 'os' is not defined

In [ ]:
extracted_data = []

for result in results:
    for item in result['data']:
        if (
            item['location'] is not None and 
            item['latestReceivedMessage'] is not None and 
            item['latestReceivedMessage']['rawData'] is not None
        ):
            raw_data = item['latestReceivedMessage']['rawData']
            if "rxInfo" in raw_data and raw_data.get('deviceInfo', {}).get('applicationName'):
                for rx_info in raw_data['rxInfo']:
                    extracted_item = {
                        'app_name': raw_data.get('deviceInfo', {}).get('applicationName', None),
                        'device_id': item['id'],
                        'device_name': item['name'],
                        'device_latitude': item['location']['coordinates'][1],
                        'device_longitude': item['location']['coordinates'][0],
                        'gateway_id_nr': item['latestReceivedMessage']['id'],
                        'last_update': item['latestReceivedMessage']['updatedAt'],
                        'gateway_name': rx_info.get('name', None),
                        'snr': rx_info.get('snr', None),
                        'rssi': rx_info.get('rssi', None),
                        'gatewayID': rx_info.get('gatewayId', None),
                        'latitude': rx_info['location'].get('latitude', None) if rx_info.get('location') else None,
                        'longitude': rx_info['location'].get('longitude', None) if rx_info.get('location') else None,
                        'spreadingFactor': raw_data.get('txInfo', {}).get('modulation', {}).get('lora', {}).get('spreadingFactor', None),
                    }
                    extracted_data.append(extracted_item)
df = pd.DataFrame(extracted_data)

urls = [
  f"{base_url}/application?limit=99999&offset=0",
  f"{base_url}/application?limit=99999&offset=0"
]

api_keys = [os2iot_api_1, os2iot_api_2]

all_data = []

for url, api_key in zip(urls, api_keys):
    headers = {
        'x-api-key': api_key
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = json.loads(response.text)
        all_data.extend(data['data'])
    else:
        print(f"Failed to fetch data from {url}")

names_and_ids = [(item['id'], item['name']) for item in all_data]

app = pd.DataFrame(names_and_ids, columns=['app_id', 'app_name'])

df['app_name'] = df['app_name'].str.replace('os2iot-', '').str.strip()
app['app_name'] = app['app_name'].str.strip()

df = df.merge(app, on='app_name', how='left')

df['app_id'] = df['app_id'].fillna(0).astype(int)

df['url'] = df.apply(lambda row: f"https://prod.os2iot.kmd.dk/applications/{row['app_id']}/iot-device/{row['device_id']}/details", axis=1)

In [ ]:
#df = df[(df['device_latitude'] != 0) & (df['device_longitude'] != 0)]

def create_linestring(row):
    device_coords = (row['device_longitude'], row['device_latitude'])
    gateway_coords = (row['longitude'], row['latitude'])
    return LineString([device_coords, gateway_coords])

df['wkt'] = df.apply(create_linestring, axis=1)

gdf = gpd.GeoDataFrame(df, geometry='wkt')
gdf['device_point'] = gdf.apply(lambda row: Point(row['device_longitude'], row['device_latitude']), axis=1)
gdf['gateway_point'] = gdf.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)

gdf['distance_m'] = gdf['wkt'].length

#gdf = gdf[['device_id', 'gateway_id_nr', 'distance_m', 'wkt', 'device_point', 'gateway_point', 'device_longitude', 'device_latitude']]
gdf = gdf.drop(columns= ['device_latitude', 'device_longitude', 'latitude', 'longitude'])
#gdf = gdf[(gdf['device_point'] != 0) & (gdf['gateway_point'] != 0)]
gdf = gdf.sort_values('last_update', ascending=False).reset_index()

def extract_coordinates(point):
    return f"{point.y}, {point.x}"

gdf['device_loc'] = gdf['device_point'].apply(extract_coordinates)

def classify_signal(rssi, snr, sf):
    thresholds = {
        7.0: {'RSSI': {'Bad': -117, 'Average': -107}, 'SNR': {'Bad': 0, 'Average': 10}},
        8.0: {'RSSI': {'Bad': -120, 'Average': -110}, 'SNR': {'Bad': -5, 'Average': 5}},
        9.0: {'RSSI': {'Bad': -123, 'Average': -113}, 'SNR': {'Bad': -8, 'Average': 2}},
        10.0: {'RSSI': {'Bad': -127, 'Average': -117}, 'SNR': {'Bad': -10, 'Average': 0}},
        11.0: {'RSSI': {'Bad': -129, 'Average': -119}, 'SNR': {'Bad': -12, 'Average': -2}},
        12.0: {'RSSI': {'Bad': -130, 'Average': -120}, 'SNR': {'Bad': -15, 'Average': -5}},
    }
    
    try:
        RSSI = float(rssi)
        SNR = float(snr)
        SF = float(sf)
    except (ValueError, TypeError):  
        return 0 # "Unknown"

    if pd.isna(SF) or SF not in thresholds:
        return 0 # "Unknown"

    rssi_thresholds = thresholds[SF]['RSSI']
    snr_thresholds = thresholds[SF]['SNR']

    if RSSI < rssi_thresholds['Bad']:
        rssi_class = 0
    elif RSSI < rssi_thresholds['Average']:
        rssi_class = 1
    else:
        rssi_class = 2

    if SNR < snr_thresholds['Bad']:
        snr_class = 0
    elif SNR < snr_thresholds['Average']:
        snr_class = 1
    else:
        snr_class = 2

    if rssi_class == 0 or snr_class == 0:
        return 0

    combined_score = rssi_class + snr_class
    return combined_score

gdf[['rssi', 'snr', 'spreadingFactor']] = gdf[['rssi', 'snr', 'spreadingFactor']].apply(pd.to_numeric, errors='coerce')
gdf['signal_quality'] = gdf.apply(lambda row: classify_signal(row['rssi'], row['snr'], row['spreadingFactor']), axis=1)

In [ ]:
gdf_device_id = gdf[gdf['device_id'] == 4232]
print(gdf_device_id['url'].iloc[0])

https://prod.os2iot.kmd.dk/applications/213/iot-device/4232/details


In [ ]:
gdf = gdf.sort_values(by=['app_name', 'device_name'])

gdf.to_csv(os.getcwd() + '/iot_devices.csv', encoding='utf-8')

In [ ]:
gdf

,index,app_name,device_id,device_name,gateway_id_nr,last_update,gateway_name,snr,rssi,gatewayID,spreadingFactor,app_id,url,wkt,device_point,gateway_point,distance_m,device_loc,signal_quality
1362,1418,0e69ef4e-71dc-4c09-a1d7-52da4db366dc,3147,0080E115002B29C5,2225,2025-03-19T23:37:59.096Z,None,-8.5,-121,7276ff002e060a9d,12,0,https://prod.os2iot.kmd.dk/applications/0/iot-...,"LINESTRING (10.19377 56.14536, 10.19843 56.14557)",POINT (10.19377 56.14536),POINT (10.19843 56.14557),0.004658,"56.145362854, 10.19377327",2
1474,1356,0e69ef4e-71dc-4c09-a1d7-52da4db366dc,2888,0080E115002B2ADC,1833,2025-02-21T18:18:09.587Z,None,-12.2,-113,7076ff00640524c6,12,0,https://prod.os2iot.kmd.dk/applications/0/iot-...,"LINESTRING (10.20115 56.14526, 10.19853 56.14428)",POINT (10.20115 56.14526),POINT (10.19853 56.14428),0.002801,"56.145256042, 10.201154709",3
1475,1355,0e69ef4e-71dc-4c09-a1d7-52da4db366dc,2888,0080E115002B2ADC,1833,2025-02-21T18:18:09.587Z,None,-16.0,-107,7276ff002e060a9d,12,0,https://prod.os2iot.kmd.dk/applications/0/iot-...,"LINESTRING (10.20115 56.14526, 10.19841 56.14557)",POINT (10.20115 56.14526),POINT (10.19841 56.14557),0.002761,"56.145256042, 10.201154709",0
1096,1042,0e69ef4e-71dc-4c09-a1d7-52da4db366dc,2497,0080E115002B2C45,1626,2025-03-20T12:36:36.217Z,None,-14.0,-105,7276ff002e060a9d,12,0,https://prod.os2iot.kmd.dk/applications/0/iot-...,"LINESTRING (10.20108 56.14711, 10.19843 56.14551)",POINT (10.20108 56.14711),POINT (10.19843 56.14551),0.003091,"56.147106171, 10.20108223",3
1254,1010,0e69ef4e-71dc-4c09-a1d7-52da4db366dc,2467,0080E115002B2C4F,1603,2025-03-20T11:20:59.067Z,None,-18.0,-109,7276ff002e060a9d,12,0,https://prod.os2iot.kmd.dk/applications/0/iot-...,"LINESTRING (10.19247 56.14419, 10.19841 56.14558)",POINT (10.19247 56.14419),POINT (10.19841 56.14558),0.006098,"56.144187927, 10.192473412",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,1569,aa205d24-faf6-4d41-ba58-b7eff32b2948,3649,KNAPPEN,2648,2025-03-11T14:21:05.483Z,None,15.0,-89,7276ff002e050616,12,0,https://prod.os2iot.kmd.dk/applications/0/iot-...,"LINESTRING (10.21385 56.1533, 10.21335 56.15332)",POINT (10.21385 56.1533),POINT (10.21335 56.15332),0.000507,"56.15330316, 10.213851929",4
899,1517,aa205d24-faf6-4d41-ba58-b7eff32b2948,3908,Milesight E-Ink D168990,2806,2025-03-20T13:03:39.371Z,None,-8.2,-104,0016c001f117e482,10,0,https://prod.os2iot.kmd.dk/applications/0/iot-...,"LINESTRING (10.21381 56.15322, 10.21394 56.15352)",POINT (10.21381 56.15322),POINT (10.21394 56.15352),0.000321,"56.153223, 10.2138149",3
900,1519,aa205d24-faf6-4d41-ba58-b7eff32b2948,3908,Milesight E-Ink D168990,2806,2025-03-20T13:03:39.371Z,None,-9.0,-88,7276ff002e050716,10,0,https://prod.os2iot.kmd.dk/applications/0/iot-...,"LINESTRING (10.21381 56.15322, 10.21416 56.15348)",POINT (10.21381 56.15322),POINT (10.21416 56.15348),0.000431,"56.153223, 10.2138149",3
901,1518,aa205d24-faf6-4d41-ba58-b7eff32b2948,3908,Milesight E-Ink D168990,2806,2025-03-20T13:03:39.371Z,None,14.0,-74,7276ff002e050616,10,0,https://prod.os2iot.kmd.dk/applications/0/iot-...,"LINESTRING (10.21381 56.15322, 10.21335 56.15332)",POINT (10.21381 56.15322),POINT (10.21335 56.15332),0.000479,"56.153223, 10.2138149",4
